# $merge的基本操作

### 範例01-If the output collection does not exist, the $merge creates the collection.

In [ ]:
db.getSiblingDB("zoo").salaries.aggregate([
    {
        $group: {
            _id: { fiscal_year: "$fiscal_year", dept: "$dept"},
            salaries: { $sum: "$salary" }
        }
    },
    {
        $merge: {
            into: { db: "reporting", coll: "budgets" },
            on: "_id",
            whenMatched: "replace",
            whenNotMatched: "insert"
        }
    }
])

### 範例02-延續範例01

In [ ]:
db.getSiblingDB("zoo").salaries.aggregate([
    {
        $match: {
            fiscal_year: { $gte: 2019 }
        }
    },
    {
        $group: {
            _id: { fiscal_year: "$fiscal_year", dept: "$dept"},
            salaries: { $sum: "$salary" }
        }
    },
    {
        $merge: {
            into: { db: "reporting", coll: "budgets" },
            on: "_id",
            whenMatched: "replace",
            whenNotMatched: "insert"
        }
    }
])

### 範例03-Only insert new data

In [ ]:
db.getSiblingDB("zoo").salaries.aggregate([
    {
        $match: {
            fiscal_year: 2019
        }
    },
    {
        $group: {
            _id: { fiscal_year: "$fiscal_year", dept: "$dept" }, 
            employees: { $push: "$employee" } 
        }
    },
    {
        $project: {
            _id: 0, 
            dept: "$_id.dept", 
            fiscal_year: "$_id.fiscal_year", 
            employees: 1 
        }
    },
    {
        $merge: {
            into : { db: "reporting", coll: "orgArchive" }, 
            on: [ "dept", "fiscal_year" ], 
            whenMatched: "fail" 
        }
    }
])

### 範例04-Merge Results from Multiple Collections

In [ ]:
db.purchaseorders.aggregate([
    {
        $group: {
            _id: "$quarter", 
            purchased: { $sum: "$qty" }
        }
    },
    {
        $merge: {
            into: "quarterlyreport",
            on: "_id",
            whenMatched: "merge",
            whenNotMatched: "insert"

        }
    }
])

In [ ]:
db.reportedsales.aggregate([
    {
        $group: {
            _id: "$quarter", 
            sales: { $sum: "$qty" }
        }
    },
    {
        $merge: {
            into: "quarterlyreport",
            on: "_id",
            whenMatched: "merge",
            whenNotMatched: "insert"

        }
    }
])

### 範例05-Use the Pipeline to Customize the Merge

In [ ]:
db.votes.aggregate([
   { $match: { date: { $gte: new Date("2019-05-07"), $lt: new Date("2019-05-08") } } },
   { $project: { _id: { $dateToString: { format: "%Y-%m", date: "$date" } }, thumbsup: 1, thumbsdown: 1 } },
   { $merge: {
         into: "monthlytotals",
         on: "_id",
         whenMatched:  [
            { $addFields: {
                thumbsup: { $add:[ "$thumbsup", "$$new.thumbsup" ] },
                thumbsdown: { $add: [ "$thumbsdown", "$$new.thumbsdown" ] }
            } } ],
         whenNotMatched: "insert"
   } }
])